# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 5 2023 2:19PM,254384,10,Enova-11169,Emerginnova,Released
1,Jan 5 2023 2:19PM,254384,10,Enova-11170,Emerginnova,Released
2,Jan 5 2023 2:19PM,254384,10,Enova-11171,Emerginnova,Released
3,Jan 5 2023 2:18PM,254383,10,SNAP15231,"Snap Medical Industries, LLC",Released
4,Jan 5 2023 2:18PM,254383,10,SNAP15232,"Snap Medical Industries, LLC",Released
5,Jan 5 2023 2:08PM,254381,10,0086281604,ISDIN Corporation,Released
6,Jan 5 2023 2:08PM,254381,10,0086281606,ISDIN Corporation,Released
7,Jan 5 2023 2:08PM,254381,10,0086281605,ISDIN Corporation,Released
8,Jan 5 2023 2:08PM,254381,10,0086281613,ISDIN Corporation,Released
9,Jan 5 2023 2:08PM,254381,10,0086280788,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,254379,Released,2
29,254380,Released,37
30,254381,Released,42
31,254383,Released,2
32,254384,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254379,NaN,NaN,2.0
254380,NaN,NaN,37.0
254381,NaN,NaN,42.0
254383,NaN,NaN,2.0
254384,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254302,0.0,1.0,1.0
254304,0.0,1.0,0.0
254307,0.0,1.0,5.0
254312,16.0,19.0,1.0
254319,0.0,0.0,50.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254302,0,1,1
254304,0,1,0
254307,0,1,5
254312,16,19,1
254319,0,0,50


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254302,0,1,1
1,254304,0,1,0
2,254307,0,1,5
3,254312,16,19,1
4,254319,0,0,50


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254302,,1,1
1,254304,,1,
2,254307,,1,5
3,254312,16,19,1
4,254319,,,50


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 5 2023 2:19PM,254384,10,Emerginnova
3,Jan 5 2023 2:18PM,254383,10,"Snap Medical Industries, LLC"
5,Jan 5 2023 2:08PM,254381,10,ISDIN Corporation
47,Jan 5 2023 2:08PM,254380,10,ISDIN Corporation
84,Jan 5 2023 2:08PM,254377,10,ISDIN Corporation
88,Jan 5 2023 2:08PM,254375,10,ISDIN Corporation
120,Jan 5 2023 2:08PM,254373,10,ISDIN Corporation
155,Jan 5 2023 2:08PM,254372,10,ISDIN Corporation
188,Jan 5 2023 2:07PM,254371,10,ISDIN Corporation
229,Jan 5 2023 2:06PM,254370,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 5 2023 2:19PM,254384,10,Emerginnova,,,3
1,Jan 5 2023 2:18PM,254383,10,"Snap Medical Industries, LLC",,,2
2,Jan 5 2023 2:08PM,254381,10,ISDIN Corporation,,,42
3,Jan 5 2023 2:08PM,254380,10,ISDIN Corporation,,,37
4,Jan 5 2023 2:08PM,254377,10,ISDIN Corporation,,,4
5,Jan 5 2023 2:08PM,254375,10,ISDIN Corporation,,,32
6,Jan 5 2023 2:08PM,254373,10,ISDIN Corporation,,,35
7,Jan 5 2023 2:08PM,254372,10,ISDIN Corporation,,,33
8,Jan 5 2023 2:07PM,254371,10,ISDIN Corporation,,,41
9,Jan 5 2023 2:06PM,254370,10,ISDIN Corporation,,,55


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 5 2023 2:19PM,254384,10,Emerginnova,3,,
1,Jan 5 2023 2:18PM,254383,10,"Snap Medical Industries, LLC",2,,
2,Jan 5 2023 2:08PM,254381,10,ISDIN Corporation,42,,
3,Jan 5 2023 2:08PM,254380,10,ISDIN Corporation,37,,
4,Jan 5 2023 2:08PM,254377,10,ISDIN Corporation,4,,
5,Jan 5 2023 2:08PM,254375,10,ISDIN Corporation,32,,
6,Jan 5 2023 2:08PM,254373,10,ISDIN Corporation,35,,
7,Jan 5 2023 2:08PM,254372,10,ISDIN Corporation,33,,
8,Jan 5 2023 2:07PM,254371,10,ISDIN Corporation,41,,
9,Jan 5 2023 2:06PM,254370,10,ISDIN Corporation,55,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 5 2023 2:19PM,254384,10,Emerginnova,3,,
1,Jan 5 2023 2:18PM,254383,10,"Snap Medical Industries, LLC",2,,
2,Jan 5 2023 2:08PM,254381,10,ISDIN Corporation,42,,
3,Jan 5 2023 2:08PM,254380,10,ISDIN Corporation,37,,
4,Jan 5 2023 2:08PM,254377,10,ISDIN Corporation,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 5 2023 2:19PM,254384,10,Emerginnova,3.0,NaN,NaN
1,Jan 5 2023 2:18PM,254383,10,"Snap Medical Industries, LLC",2.0,NaN,NaN
2,Jan 5 2023 2:08PM,254381,10,ISDIN Corporation,42.0,NaN,NaN
3,Jan 5 2023 2:08PM,254380,10,ISDIN Corporation,37.0,NaN,NaN
4,Jan 5 2023 2:08PM,254377,10,ISDIN Corporation,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 5 2023 2:19PM,254384,10,Emerginnova,3.0,0.0,0.0
1,Jan 5 2023 2:18PM,254383,10,"Snap Medical Industries, LLC",2.0,0.0,0.0
2,Jan 5 2023 2:08PM,254381,10,ISDIN Corporation,42.0,0.0,0.0
3,Jan 5 2023 2:08PM,254380,10,ISDIN Corporation,37.0,0.0,0.0
4,Jan 5 2023 2:08PM,254377,10,ISDIN Corporation,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4069967,411.0,0.0,0.0
15,762965,4.0,20.0,16.0
19,1271627,56.0,3.0,0.0
20,508684,2.0,0.0,0.0
21,763041,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4069967,411.0,0.0,0.0
1,15,762965,4.0,20.0,16.0
2,19,1271627,56.0,3.0,0.0
3,20,508684,2.0,0.0,0.0
4,21,763041,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,411.0,0.0,0.0
1,15,4.0,20.0,16.0
2,19,56.0,3.0,0.0
3,20,2.0,0.0,0.0
4,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,411.0
1,15,Released,4.0
2,19,Released,56.0
3,20,Released,2.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,0.0,16.0,0.0,0.0,0.0
Executing,0.0,20.0,3.0,0.0,0.0
Released,411.0,4.0,56.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,0.0,16.0,0.0,0.0,0.0
1,Executing,0.0,20.0,3.0,0.0,0.0
2,Released,411.0,4.0,56.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,0.0,16.0,0.0,0.0,0.0
1,Executing,0.0,20.0,3.0,0.0,0.0
2,Released,411.0,4.0,56.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()